In [6]:
# Run this to install latest version of PyTorch and Lightning

!pip3 install pytorch-lightning
!pip3 install torch
!pip3 install transformers
!pip3 install datasets
!pip3 install emoji # to convert emoticons into their text versions

     |████████████████████████████████| 525 kB 27.3 MB/s 
     |████████████████████████████████| 132 kB 77.3 MB/s 
     |████████████████████████████████| 596 kB 44.5 MB/s 
     |████████████████████████████████| 332 kB 56.9 MB/s 
     |████████████████████████████████| 829 kB 50.1 MB/s 
     |████████████████████████████████| 1.1 MB 30.4 MB/s 
     |████████████████████████████████| 160 kB 72.6 MB/s 
     |████████████████████████████████| 271 kB 50.6 MB/s 
     |████████████████████████████████| 192 kB 76.3 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=6663b4caf8b618baacbb89294c6b250fc71c442d8084da288de1bbd01c092c53
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: future
    Found exi

In [2]:
from google.colab import drive 
drive.mount('/content/drive')
%cd drive/MyDrive/XAI_gitlab/conecept-x-for-emotion-detection-2/

Mounted at /content/drive


In [7]:
from Model.EmoClassifier import TextClassifierModule
from Data.Preprocessing import foo
import os

In [ ]:
# Mount gdrive support and set working directory for future use

from google.colab import drive
drive.mount('/content/drive')

os.chdir('/content/drive/My Drive/emotion_detection')
os.getcwd() # use this directory as root, all logs and model checkpoints are saved here

In [ ]:
combined_data_path = "{}/datasets/combined_data.csv".format(os.getcwd())

In [ ]:
data_module,tokenizer,classes = foo(  data_path=combined_data_path,
                                label_name="broadEmo",
                                dataset_names=["Empathetic Dialogues"], 
                                split_train_val_test=[0.8,0.1,0.1], 
                                max_len=60, 
                                tokenizer="BertweetTokenizer", 
                                batch_size=32, 
                                RANDOM_SEED=5)

In [ ]:
# Defining network hpyeprapameters here. Hyperparameters can include anything and will be saved in model checkpoints
# These hyperparameters will be avaialable in the neural network module.

hparams = {
  'freeze_feature_extractor': True,
  'freeze_until_layer': 8, 
  'optimizer': 'Adam',
  'featuer_extactor_lr': 3e-5,
  'learning_rate': 3e-5,
  'weight_decay': 1e-5,
  'batch_size': 32, # needed in model for logging,
}

In [ ]:
model = TextClassifierModule(len(classes), **hparams).float()

In [ ]:
#summarize(model, max_depth=-1) # obtain full summary of the model

# TensorBoard Logging

TensorBoard provides a good user interface to track training process.
It can be used to track metric curves during traning and compare several versions of models at the same time. It also allows to view model architectures with input and output at each layer.

# Training

Pytorch lightning Trainer takes in the data module and lightning module
and performs the trainig loop automatically. It provides a bunch of useful
callback to manage the training session (eg. early stopping, gradient tracking, weight averaging, etc.) Trainer can automatically track validation metrics and save best model checkpoints. Trainer removes the overhead of manually tracking every bit of training loop and eliminates boilerplate

In [ ]:
''' 
Pytorch Lightning provides a Trainer class which handles training loop.
The trainer automatically performs validation and training steps while also
logging key metrics which enables effective training.

The trainer takes in data modules and a model and automatically sets up the training loop

The trainier automatically handles GPU or TPU training without the need of
manually casting tensors to device.

The trainer saves model checkpoints so that best models can later be recovered

The trainer supports wide variety of options which make model training more efficient
and fast (16 bit precision, debugging, early stopping, etc.)

Full manual can be found here: https://pytorch-lightning.readthedocs.io/en/latest/common/trainer.html
'''
# Getting a warning about ambigous batch size
# The warning is nothing serious, it happens because pytorch lightning
# does not handle dictionaries as data inputs well. In reality training loop
# works properly

# Warning: https://github.com/PyTorchLightning/pytorch-lightning/issues/10349 
import warnings
warnings.filterwarnings('ignore')

classifier_logger = TensorBoardLogger(save_dir='lightning_logs', log_graph = True)

checkpoint_callback = ModelCheckpoint(
  monitor="val_loss",
  dirpath="models/",
  filename="emotion-recognizer",
  save_top_k=1,
  mode="min",
)
# !!TRAINING ON TPU causes exceptions for some reason. We have to stick to GPU for some time :(
# It seem bertweet model is not compatible with TPU-s. This is stated in tensorbaord, under TPU compatability (0%)

trainer = pl.Trainer(
  # overfit_batches=1, # debug option, overfits the given proportion of the whole data
  track_grad_norm=2, # debug option, tracks gradient norms in tensorboard
  default_root_dir=os.getcwd(), # The directory to save and log training results
  max_epochs=30,
  gpus=1 if torch.cuda.is_available() else None, # Uncomment to use GPU training when available
  # tpu_cores=8, # Uncomment to train on TPU when it is available. Make sure to enable it before running with this flag
  val_check_interval=0.1, # validate 10 times per epoch, frequent validation is helpful
  logger=classifier_logger, # Logger options to track training
  callbacks=[EarlyStopping(monitor="val_loss", patience=10), checkpoint_callback],
  # checkpoint_callback=False, # toggles checkpointing, might be good to avoid when debugging
  # precision=16, # enable 16 bit precision. Allows multiple times faster training. Works only on GPU so far
)

trainer.fit(model, data_module)